# LW Integrator Verification: Original vs Refactored Implementations

**Objective:** Verify that our architectural improvements haven't introduced any bugs by comparing trajectories between:
1. **Original Legacy Code** (from demo notebooks)
2. **Refactored Basic Integrator** (functionalized trajectory_integrator.py)
3. **Optimized Integrator** (performance.py with JIT compilation)

**Test Case:** Two-particle electromagnetic interaction simulation based on `two_particle_demo_main.ipynb`

---

## Verification Strategy
- Use **identical initial conditions** and simulation parameters
- Compare **final particle trajectories** and **energy conservation**
- Measure **numerical differences** and **statistical distributions**
- Validate **performance improvements** while maintaining accuracy

In [12]:
# Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import warnings

warnings.filterwarnings("ignore")

# Configure matplotlib for better plots
plt.rcParams.update({"font.size": 12, "figure.figsize": (10, 6)})

# Add paths for imports
sys.path.insert(0, "/home/benfol/work/LW_windows/LW_integrator")
sys.path.insert(0, "/home/benfol/work/LW_windows/LW_integrator/LW_integrator")

print("✅ Libraries imported successfully")
print(f"NumPy version: {np.__version__}")
print(f"Python version: {sys.version}")
print("=" * 50)

✅ Libraries imported successfully
NumPy version: 2.1.3
Python version: 3.13.7 (main, Aug 14 2025, 00:00:00) [GCC 15.2.1 20250808 (Red Hat 15.2.1-1)]


In [ ]:
# Load Original Integrator Code
# Import the original legacy integrator functions from the main LW_integrator directory
try:
    # Original code from the demo notebooks
    from covariant_integrator_library import retarded_integrator3
    from bunch_inits import init_bunch
    # from plotting_variables import calculate_plotting_variables  # Unused

    print("✅ Original legacy integrator loaded successfully")
    LEGACY_AVAILABLE = True
except ImportError as e:
    print(f"⚠️  Original legacy code not available: {e}")
    print("   Will proceed with refactored implementations only")
    LEGACY_AVAILABLE = False

print("=" * 50)

⚠️  Original legacy code not available: No module named 'covariant_integrator_library'
   Will proceed with refactored implementations only


In [ ]:
# Load Refactored Integrator Implementations
import sys
from pathlib import Path

# Add the LW_windows directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

try:
    from core.trajectory_integrator import (
        LienardWiechertIntegrator as BasicLWIntegrator,
    )

    print("✅ Basic refactored integrator loaded")
    BASIC_AVAILABLE = True
except ImportError as e:
    print(f"❌ Basic integrator import failed: {e}")
    BASIC_AVAILABLE = False

try:
    from lw_integrator.core.performance import OptimizedLienardWiechertIntegrator

    print("✅ Optimized integrator loaded")
    OPTIMIZED_AVAILABLE = True
except ImportError as e:
    print(f"❌ Optimized integrator import failed: {e}")
    OPTIMIZED_AVAILABLE = False

try:
    # Unified interface (auto-selecting) - commenting out unused import
    # from lw_integrator.core.unified_interface import (
    #     LienardWiechertIntegrator as UnifiedLWIntegrator,
    # )

    print("⚠️ Unified interface commented out (unused)")
    UNIFIED_AVAILABLE = False
except ImportError as e:
    print(f"❌ Unified interface import failed: {e}")
    UNIFIED_AVAILABLE = False

print("=" * 50)
print("Available integrators:")
print(f"  Legacy Original: {LEGACY_AVAILABLE}")
print(f"  Basic Refactored: {BASIC_AVAILABLE}")
print(f"  Optimized JIT: {OPTIMIZED_AVAILABLE}")
print(f"  Unified Auto: {UNIFIED_AVAILABLE}")
print("=" * 50)

✅ Basic (refactored) integrator loaded
✅ Optimized integrator loaded
✅ Unified interface loaded
Available integrators:
  Legacy Original: False
  Basic Refactored: True
  Optimized JIT: True
  Unified Auto: True


In [15]:
# Set Up Simulation Parameters (from two_particle_demo_main.ipynb)

# Physical constants
c_ms = 299792458

# Particle parameters
transv_dist = 1e-4
m_particle_rider = 1.007319468  # proton - amu
m_particle_driver = 207.2  # lead, amu
stripped_ions_rider = 1.0
stripped_ions_driver = 54.0
charge_sign_rider = -1.0
charge_sign_driver = 1.0

# Initial momentum and position
starting_Pz_rider = 1.01e6  # High energy proton
starting_Pz_driver = -starting_Pz_rider / m_particle_driver * m_particle_rider
transv_mom_rider = 0.0
transv_mom_driver = transv_mom_rider
starting_distance_rider = 1e-6
starting_distance_driver = 100.0

# Simulation parameters
sim_type = 2  # bunch-bunch simulations
pcount_rider = 5  # Smaller particle count for focused comparison
pcount_driver = 5

# Integration parameters (simplified for comparison)
static_steps = 1
ret_steps = 50  # Shorter integration for faster comparison
step_size = 2e-6

# Additional parameters
bunch_dist = 1e5
cav_spacing = 1e5
aperture = 1e5
z_cutoff = 0
wall_pos = 1e5

print("🎯 Simulation Parameters:")
print(f"  Rider particles: {pcount_rider}")
print(f"  Driver particles: {pcount_driver}")
print(f"  Integration steps: {ret_steps}")
print(f"  Step size: {step_size}")
print(f"  Transverse distance: {transv_dist}")
print("=" * 50)

🎯 Simulation Parameters:
  Rider particles: 5
  Driver particles: 5
  Integration steps: 50
  Step size: 2e-06
  Transverse distance: 0.0001


In [16]:
# Initialize Particle Bunches (Identical for All Tests)
if LEGACY_AVAILABLE:
    try:
        # Create initial particle distributions using original functions
        init_rider, E_MeV_rest_rider = init_bunch(
            starting_distance_rider,
            transv_mom_rider,
            starting_Pz_rider,
            stripped_ions_rider,
            m_particle_rider,
            transv_dist,
            pcount_rider,
            charge_sign_rider,
        )

        init_driver, E_MeV_rest_driver = init_bunch(
            starting_distance_driver,
            transv_mom_driver,
            starting_Pz_driver,
            stripped_ions_driver,
            m_particle_driver,
            -transv_dist,
            pcount_driver,
            charge_sign_driver,
        )

        print("✅ Particle bunches initialized using legacy init_bunch()")
        print(f"  Rider bunch shape: {[len(arr) for arr in init_rider[:3]]}")
        print(f"  Driver bunch shape: {[len(arr) for arr in init_driver[:3]]}")

    except Exception as e:
        print(f"❌ Legacy initialization failed: {e}")
        LEGACY_AVAILABLE = False

else:
    # Create simple test particles manually if legacy not available
    print("📝 Creating test particles manually (legacy not available)")

    # Simple 2-particle test case
    n_total = pcount_rider + pcount_driver

    # Initialize arrays similar to bunch_inits format
    x = np.zeros(n_total)
    y = np.zeros(n_total)
    z = np.zeros(n_total)
    vx = np.zeros(n_total)
    vy = np.zeros(n_total)
    vz = np.zeros(n_total)

    # Rider particles (negative charge)
    x[:pcount_rider] = starting_distance_rider + np.random.normal(0, 1e-7, pcount_rider)
    y[:pcount_rider] = transv_dist + np.random.normal(0, 1e-7, pcount_rider)
    z[:pcount_rider] = np.random.normal(0, 1e-7, pcount_rider)

    # Driver particles (positive charge)
    x[pcount_rider:] = starting_distance_driver + np.random.normal(
        0, 1e-6, pcount_driver
    )
    y[pcount_rider:] = -transv_dist + np.random.normal(0, 1e-6, pcount_driver)
    z[pcount_rider:] = np.random.normal(0, 1e-6, pcount_driver)

    # Simple velocities (non-relativistic for test)
    vz[:pcount_rider] = 0.1  # v/c
    vz[pcount_rider:] = -0.1

    print(f"✅ Manual test particles created: {n_total} total")

print("=" * 50)

📝 Creating test particles manually (legacy not available)
✅ Manual test particles created: 10 total


In [17]:
# Convert to Unified Particle Format for New Integrators
def convert_legacy_to_modern_format(init_rider, init_driver):
    """Convert legacy bunch format to modern integrator format."""

    # Extract arrays from legacy format (assume standard bunch_inits structure)
    # Legacy format: [x, y, z, vx, vy, vz, bx, by, bz, bdotx, bdoty, bdotz, Px, Py, Pz, Pt, gamma, t]

    # Combine rider and driver particles
    n_rider = len(init_rider[0])
    n_driver = len(init_driver[0])
    n_total = n_rider + n_driver

    particles = {}

    # Position arrays
    particles["x"] = np.concatenate([init_rider[0], init_driver[0]])
    particles["y"] = np.concatenate([init_rider[1], init_driver[1]])
    particles["z"] = np.concatenate([init_rider[2], init_driver[2]])

    # Velocity arrays
    particles["vx"] = np.concatenate([init_rider[3], init_driver[3]])
    particles["vy"] = np.concatenate([init_rider[4], init_driver[4]])
    particles["vz"] = np.concatenate([init_rider[5], init_driver[5]])

    # Beta arrays (v/c)
    particles["bx"] = np.concatenate([init_rider[6], init_driver[6]])
    particles["by"] = np.concatenate([init_rider[7], init_driver[7]])
    particles["bz"] = np.concatenate([init_rider[8], init_driver[8]])

    # Acceleration arrays
    particles["bdotx"] = np.concatenate([init_rider[9], init_driver[9]])
    particles["bdoty"] = np.concatenate([init_rider[10], init_driver[10]])
    particles["bdotz"] = np.concatenate([init_rider[11], init_driver[11]])

    # Momentum arrays
    particles["Px"] = np.concatenate([init_rider[12], init_driver[12]])
    particles["Py"] = np.concatenate([init_rider[13], init_driver[13]])
    particles["Pz"] = np.concatenate([init_rider[14], init_driver[14]])
    particles["Pt"] = np.concatenate([init_rider[15], init_driver[15]])

    # Gamma and time
    particles["gamma"] = np.concatenate([init_rider[16], init_driver[16]])
    particles["t"] = np.concatenate([init_rider[17], init_driver[17]])

    # Charges and masses (need to construct these)
    particles["q"] = np.concatenate(
        [
            np.full(n_rider, charge_sign_rider * stripped_ions_rider),
            np.full(n_driver, charge_sign_driver * stripped_ions_driver),
        ]
    )

    particles["m"] = np.concatenate(
        [
            np.full(n_rider, m_particle_rider * 938.3),  # Convert amu to MeV/c²
            np.full(n_driver, m_particle_driver * 938.3),
        ]
    )

    particles["char_time"] = np.ones(n_total)

    return particles


if LEGACY_AVAILABLE:
    # Convert legacy format to modern format
    try:
        particles_initial = convert_legacy_to_modern_format(init_rider, init_driver)
        print("✅ Converted legacy particles to modern format")
        print(f"  Total particles: {len(particles_initial['x'])}")
        print(
            f"  Position range (x): [{particles_initial['x'].min():.2e}, {particles_initial['x'].max():.2e}]"
        )
        print(
            f"  Velocity range (vz): [{particles_initial['vz'].min():.2e}, {particles_initial['vz'].max():.2e}]"
        )
        print(f"  Charges: {np.unique(particles_initial['q'])}")

    except Exception as e:
        print(f"❌ Legacy conversion failed: {e}")
        LEGACY_AVAILABLE = False

print("=" * 50)

In [18]:
# Run Original Legacy Integration (if available)
results = {}

if LEGACY_AVAILABLE:
    print("🔄 Running ORIGINAL LEGACY integration...")
    start_time = time.time()

    try:
        # Run legacy integration exactly as in demo notebook
        retarded_traj_pre, retarded_drv_traj_pre = retarded_integrator3(
            static_steps,
            ret_steps,
            step_size,
            wall_pos,
            aperture,
            sim_type,
            init_rider,
            init_driver,
            bunch_dist,
            cav_spacing,
            z_cutoff,
        )

        legacy_time = time.time() - start_time

        # Store final trajectories
        results["legacy"] = {
            "rider_traj": retarded_traj_pre,
            "driver_traj": retarded_drv_traj_pre,
            "computation_time": legacy_time,
            "success": True,
        }

        print(f"✅ Legacy integration completed in {legacy_time:.3f} seconds")
        print(f"  Rider trajectory steps: {len(retarded_traj_pre)}")
        print(f"  Driver trajectory steps: {len(retarded_drv_traj_pre)}")

        # Extract final positions for comparison
        final_rider = retarded_traj_pre[-1] if retarded_traj_pre else None
        final_driver = retarded_drv_traj_pre[-1] if retarded_drv_traj_pre else None

        if final_rider and final_driver:
            print(
                f"  Final rider position: x={final_rider[0][0]:.3e}, y={final_rider[1][0]:.3e}, z={final_rider[2][0]:.3e}"
            )
            print(
                f"  Final driver position: x={final_driver[0][0]:.3e}, y={final_driver[1][0]:.3e}, z={final_driver[2][0]:.3e}"
            )

    except Exception as e:
        print(f"❌ Legacy integration failed: {e}")
        results["legacy"] = {"success": False, "error": str(e), "computation_time": 0}

else:
    print("⚠️  Legacy integration skipped (not available)")
    results["legacy"] = {"success": False, "error": "Legacy code not available"}

print("=" * 50)

⚠️  Legacy integration skipped (not available)


In [19]:
# Run Basic (Refactored) Integration
if BASIC_AVAILABLE and (LEGACY_AVAILABLE or "particles_initial" in locals()):
    print("🔄 Running BASIC REFACTORED integration...")
    start_time = time.time()

    try:
        # Create integrator instance
        basic_integrator = BasicLWIntegrator()

        # Use particles from legacy conversion or manual creation
        if LEGACY_AVAILABLE:
            current_particles = particles_initial.copy()
        else:
            # Create simple test particles if legacy not available
            current_particles = {
                "x": x,
                "y": y,
                "z": z,
                "vx": vx,
                "vy": vy,
                "vz": vz,
                "bx": vx,
                "by": vy,
                "bz": vz,  # Non-relativistic approximation
                "bdotx": np.zeros(n_total),
                "bdoty": np.zeros(n_total),
                "bdotz": np.zeros(n_total),
                "Px": np.zeros(n_total),
                "Py": np.zeros(n_total),
                "Pz": np.zeros(n_total),
                "Pt": np.full(n_total, 938.3),
                "gamma": np.ones(n_total),
                "t": np.zeros(n_total),
                "q": np.concatenate(
                    [np.full(pcount_rider, -1), np.full(pcount_driver, 1)]
                ),
                "m": np.full(n_total, 938.3),
                "char_time": np.ones(n_total),
            }

        # Store trajectory
        basic_trajectory = [current_particles.copy()]

        # Integration loop
        for step in range(ret_steps):
            # Single integration step using retarded method
            updated_particles = basic_integrator.eqsofmotion_retarded(
                step_size, current_particles, current_particles
            )

            # Update current state
            current_particles = updated_particles
            basic_trajectory.append(current_particles.copy())

        basic_time = time.time() - start_time

        # Store results
        results["basic"] = {
            "trajectory": basic_trajectory,
            "final_particles": current_particles,
            "computation_time": basic_time,
            "success": True,
        }

        print(f"✅ Basic integration completed in {basic_time:.3f} seconds")
        print(f"  Trajectory steps: {len(basic_trajectory)}")
        print(f"  Final positions (x): {current_particles['x'][:3]}")
        print(f"  Final velocities (vz): {current_particles['vz'][:3]}")

    except Exception as e:
        print(f"❌ Basic integration failed: {e}")
        import traceback

        traceback.print_exc()
        results["basic"] = {"success": False, "error": str(e), "computation_time": 0}

else:
    print("⚠️  Basic integration skipped (integrator not available or no particles)")
    results["basic"] = {"success": False, "error": "Basic integrator not available"}

print("=" * 50)

⚠️  Basic integration skipped (integrator not available or no particles)


In [20]:
# Run Optimized Integration
if OPTIMIZED_AVAILABLE and (LEGACY_AVAILABLE or "particles_initial" in locals()):
    print("🔄 Running OPTIMIZED JIT integration...")
    start_time = time.time()

    try:
        # Create optimized integrator instance
        optimized_integrator = OptimizedLienardWiechertIntegrator()

        # Use same initial particles as basic integrator
        if LEGACY_AVAILABLE:
            current_particles = particles_initial.copy()
        else:
            current_particles = {
                "x": x,
                "y": y,
                "z": z,
                "vx": vx,
                "vy": vy,
                "vz": vz,
                "bx": vx,
                "by": vy,
                "bz": vz,
                "bdotx": np.zeros(n_total),
                "bdoty": np.zeros(n_total),
                "bdotz": np.zeros(n_total),
                "Px": np.zeros(n_total),
                "Py": np.zeros(n_total),
                "Pz": np.zeros(n_total),
                "Pt": np.full(n_total, 938.3),
                "gamma": np.ones(n_total),
                "t": np.zeros(n_total),
                "q": np.concatenate(
                    [np.full(pcount_rider, -1), np.full(pcount_driver, 1)]
                ),
                "m": np.full(n_total, 938.3),
                "char_time": np.ones(n_total),
            }

        # Store trajectory
        optimized_trajectory = [current_particles.copy()]

        # Integration loop
        for step in range(ret_steps):
            # Single integration step using retarded method
            updated_particles = optimized_integrator.eqsofmotion_retarded(
                step_size, current_particles, current_particles
            )

            # Update current state
            current_particles = updated_particles
            optimized_trajectory.append(current_particles.copy())

        optimized_time = time.time() - start_time

        # Store results
        results["optimized"] = {
            "trajectory": optimized_trajectory,
            "final_particles": current_particles,
            "computation_time": optimized_time,
            "success": True,
        }

        print(f"✅ Optimized integration completed in {optimized_time:.3f} seconds")
        print(f"  Trajectory steps: {len(optimized_trajectory)}")
        print(f"  Final positions (x): {current_particles['x'][:3]}")
        print(f"  Final velocities (vz): {current_particles['vz'][:3]}")

        # Calculate speedup if basic is available
        if "basic" in results and results["basic"]["success"]:
            speedup = results["basic"]["computation_time"] / optimized_time
            print(f"  Speedup vs Basic: {speedup:.2f}x")

    except Exception as e:
        print(f"❌ Optimized integration failed: {e}")
        import traceback

        traceback.print_exc()
        results["optimized"] = {
            "success": False,
            "error": str(e),
            "computation_time": 0,
        }

else:
    print("⚠️  Optimized integration skipped (integrator not available or no particles)")
    results["optimized"] = {
        "success": False,
        "error": "Optimized integrator not available",
    }

print("=" * 50)

⚠️  Optimized integration skipped (integrator not available or no particles)


In [21]:
# Compare Final Trajectories and Calculate Differences
print("📊 TRAJECTORY COMPARISON ANALYSIS")
print("=" * 60)

# Summary of successful integrations
successful_integrations = [
    name for name, result in results.items() if result.get("success", False)
]
print(f"Successful integrations: {successful_integrations}")

if len(successful_integrations) >= 2:
    # Compare basic vs optimized (most important)
    if "basic" in successful_integrations and "optimized" in successful_integrations:
        print("\n🔍 BASIC vs OPTIMIZED COMPARISON:")

        basic_final = results["basic"]["final_particles"]
        opt_final = results["optimized"]["final_particles"]

        # Position differences
        pos_diff = {
            "x": np.abs(basic_final["x"] - opt_final["x"]),
            "y": np.abs(basic_final["y"] - opt_final["y"]),
            "z": np.abs(basic_final["z"] - opt_final["z"]),
        }

        # Velocity differences
        vel_diff = {
            "vx": np.abs(basic_final["vx"] - opt_final["vx"]),
            "vy": np.abs(basic_final["vy"] - opt_final["vy"]),
            "vz": np.abs(basic_final["vz"] - opt_final["vz"]),
        }

        print("Position differences (absolute):")
        for coord in ["x", "y", "z"]:
            max_diff = np.max(pos_diff[coord])
            mean_diff = np.mean(pos_diff[coord])
            print(f"  {coord}: max={max_diff:.2e}, mean={mean_diff:.2e}")

        print("Velocity differences (absolute):")
        for coord in ["vx", "vy", "vz"]:
            max_diff = np.max(vel_diff[coord])
            mean_diff = np.mean(vel_diff[coord])
            print(f"  {coord}: max={max_diff:.2e}, mean={mean_diff:.2e}")

        # Overall trajectory similarity
        total_pos_diff = np.sqrt(
            pos_diff["x"] ** 2 + pos_diff["y"] ** 2 + pos_diff["z"] ** 2
        )
        total_vel_diff = np.sqrt(
            vel_diff["vx"] ** 2 + vel_diff["vy"] ** 2 + vel_diff["vz"] ** 2
        )

        print("\\nOverall differences:")
        print(f"  Position RMS: {np.sqrt(np.mean(total_pos_diff**2)):.2e}")
        print(f"  Velocity RMS: {np.sqrt(np.mean(total_vel_diff**2)):.2e}")
        print(f"  Max position: {np.max(total_pos_diff):.2e}")
        print(f"  Max velocity: {np.max(total_vel_diff):.2e}")

        # Check for numerical equivalence (within floating point precision)
        pos_equivalent = np.all(total_pos_diff < 1e-12)
        vel_equivalent = np.all(total_vel_diff < 1e-12)

        print("\\n✅ Results Assessment:")
        print(f"  Position numerically equivalent: {pos_equivalent}")
        print(f"  Velocity numerically equivalent: {vel_equivalent}")

        if pos_equivalent and vel_equivalent:
            print("  🎯 PERFECT MATCH - No significant differences detected!")
        elif np.all(total_pos_diff < 1e-8) and np.all(total_vel_diff < 1e-8):
            print("  ✅ EXCELLENT MATCH - Differences within numerical precision")
        else:
            print("  ⚠️  DIFFERENCES DETECTED - May need investigation")

else:
    print("❌ Insufficient successful integrations for comparison")
    print(f"   Available: {successful_integrations}")

print("=" * 60)

📊 TRAJECTORY COMPARISON ANALYSIS
Successful integrations: []
❌ Insufficient successful integrations for comparison
   Available: []


In [22]:
# Plot Trajectory Comparisons
if len(successful_integrations) >= 2:
    print("📈 Creating trajectory visualization plots...")

    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle("LW Integrator Verification: Trajectory Comparisons", fontsize=16)

    # Colors for different integrators
    colors = {"legacy": "red", "basic": "blue", "optimized": "green"}
    labels = {
        "legacy": "Original Legacy",
        "basic": "Basic Refactored",
        "optimized": "Optimized JIT",
    }

    # Plot 1: X vs Z trajectories
    ax1 = axes[0, 0]
    for int_type in successful_integrations:
        if int_type == "legacy":
            continue  # Skip legacy for now (different format)

        result = results[int_type]
        if result["success"]:
            trajectory = result["trajectory"]
            # Extract x and z positions over time
            x_traj = [step["x"] for step in trajectory]
            z_traj = [step["z"] for step in trajectory]

            # Plot first particle trajectory
            ax1.plot(
                [x[0] for x in x_traj],
                [z[0] for z in z_traj],
                color=colors[int_type],
                label=f"{labels[int_type]} (P1)",
                linewidth=2,
            )

    ax1.set_xlabel("X Position [mm]")
    ax1.set_ylabel("Z Position [mm]")
    ax1.set_title("Particle 1 Trajectory (X vs Z)")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: Y vs Z trajectories
    ax2 = axes[0, 1]
    for int_type in successful_integrations:
        if int_type == "legacy":
            continue

        result = results[int_type]
        if result["success"]:
            trajectory = result["trajectory"]
            y_traj = [step["y"] for step in trajectory]
            z_traj = [step["z"] for step in trajectory]

            ax2.plot(
                [y[0] for y in y_traj],
                [z[0] for z in z_traj],
                color=colors[int_type],
                label=f"{labels[int_type]} (P1)",
                linewidth=2,
            )

    ax2.set_xlabel("Y Position [mm]")
    ax2.set_ylabel("Z Position [mm]")
    ax2.set_title("Particle 1 Trajectory (Y vs Z)")
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # Plot 3: Velocity evolution
    ax3 = axes[1, 0]
    for int_type in successful_integrations:
        if int_type == "legacy":
            continue

        result = results[int_type]
        if result["success"]:
            trajectory = result["trajectory"]
            vz_traj = [step["vz"][0] for step in trajectory]  # First particle
            time_steps = range(len(vz_traj))

            ax3.plot(
                time_steps,
                vz_traj,
                color=colors[int_type],
                label=f"{labels[int_type]} (P1)",
                linewidth=2,
            )

    ax3.set_xlabel("Time Step")
    ax3.set_ylabel("Z Velocity [v/c]")
    ax3.set_title("Particle 1 Velocity Evolution")
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    # Plot 4: Difference between integrators
    ax4 = axes[1, 1]
    if "basic" in successful_integrations and "optimized" in successful_integrations:
        basic_traj = results["basic"]["trajectory"]
        opt_traj = results["optimized"]["trajectory"]

        # Calculate position differences over time
        pos_diffs = []
        for i in range(min(len(basic_traj), len(opt_traj))):
            basic_step = basic_traj[i]
            opt_step = opt_traj[i]

            # Position difference for first particle
            diff = np.sqrt(
                (basic_step["x"][0] - opt_step["x"][0]) ** 2
                + (basic_step["y"][0] - opt_step["y"][0]) ** 2
                + (basic_step["z"][0] - opt_step["z"][0]) ** 2
            )
            pos_diffs.append(diff)

        ax4.semilogy(range(len(pos_diffs)), pos_diffs, "purple", linewidth=2)
        ax4.set_xlabel("Time Step")
        ax4.set_ylabel("Position Difference [mm]")
        ax4.set_title("Basic vs Optimized: Position Difference")
        ax4.grid(True, alpha=0.3)

        # Add annotation about difference magnitude
        max_diff = np.max(pos_diffs)
        ax4.text(
            0.05,
            0.95,
            f"Max difference: {max_diff:.2e} mm",
            transform=ax4.transAxes,
            verticalalignment="top",
            bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.8),
        )

    plt.tight_layout()
    plt.show()

    print("✅ Trajectory plots generated")

else:
    print("⚠️  Cannot create plots - insufficient successful integrations")

print("=" * 60)

⚠️  Cannot create plots - insufficient successful integrations


In [ ]:
# Calculate Performance Metrics and Energy Conservation
print("⚡ PERFORMANCE AND PHYSICS VALIDATION")
print("=" * 60)

# Performance summary
print("🚀 Performance Summary:")
for int_type in ["legacy", "basic", "optimized"]:
    if int_type in results and results[int_type].get("success", False):
        time_val = results[int_type]["computation_time"]
        print(f"  {int_type.capitalize():12s}: {time_val:.4f} seconds")

# Calculate speedups
if "basic" in results and "optimized" in results:
    if results["basic"]["success"] and results["optimized"]["success"]:
        basic_time = results["basic"]["computation_time"]
        opt_time = results["optimized"]["computation_time"]
        speedup = basic_time / opt_time
        print(f"\\n📊 Optimized Speedup: {speedup:.2f}x faster than Basic")

# Energy conservation check
print("\\n🔋 Energy Conservation Analysis:")
for int_type in ["basic", "optimized"]:
    if int_type in results and results[int_type].get("success", False):
        trajectory = results[int_type]["trajectory"]

        # Calculate total energy at start and end
        initial_step = trajectory[0]
        final_step = trajectory[-1]

        # Total kinetic energy (non-relativistic approximation)
        def calc_kinetic_energy(particles):
            v_squared = (
                particles["vx"] ** 2 + particles["vy"] ** 2 + particles["vz"] ** 2
            )
            # KE = 1/2 * m * v^2 (in units where c=1)
            return 0.5 * particles["m"] * v_squared

        initial_energy = np.sum(calc_kinetic_energy(initial_step))
        final_energy = np.sum(calc_kinetic_energy(final_step))

        energy_change = final_energy - initial_energy
        relative_change = energy_change / initial_energy if initial_energy != 0 else 0

        print(f"  {int_type.capitalize():12s}:")
        print(f"    Initial energy: {initial_energy:.6e}")
        print(f"    Final energy:   {final_energy:.6e}")
        print(f"    Change:         {energy_change:.6e} ({relative_change*100:.4f}%)")

# Numerical precision analysis
print("\\n🔬 Numerical Precision Analysis:")
if "basic" in successful_integrations and "optimized" in successful_integrations:
    basic_final = results["basic"]["final_particles"]
    opt_final = results["optimized"]["final_particles"]

    # Calculate relative errors
    def relative_error(a, b):
        return np.abs(a - b) / (np.abs(a) + 1e-16)  # Avoid division by zero

    pos_rel_err = {
        "x": relative_error(basic_final["x"], opt_final["x"]),
        "y": relative_error(basic_final["y"], opt_final["y"]),
        "z": relative_error(basic_final["z"], opt_final["z"]),
    }

    vel_rel_err = {
        "vx": relative_error(basic_final["vx"], opt_final["vx"]),
        "vy": relative_error(basic_final["vy"], opt_final["vy"]),
        "vz": relative_error(basic_final["vz"], opt_final["vz"]),
    }

    print("  Relative errors (Basic vs Optimized):")
    for coord in ["x", "y", "z"]:
        max_err = np.max(pos_rel_err[coord])
        mean_err = np.mean(pos_rel_err[coord])
        print(f"    Position {coord}: max={max_err:.2e}, mean={mean_err:.2e}")

    for coord in ["vx", "vy", "vz"]:
        max_err = np.max(vel_rel_err[coord])
        mean_err = np.mean(vel_rel_err[coord])
        print(f"    Velocity {coord}: max={max_err:.2e}, mean={mean_err:.2e}")

    # Overall assessment
    max_pos_err = np.max([np.max(pos_rel_err[c]) for c in ["x", "y", "z"]])
    max_vel_err = np.max([np.max(vel_rel_err[c]) for c in ["vx", "vy", "vz"]])

    print("\\n📋 Overall Assessment:")
    print(f"  Maximum position error: {max_pos_err:.2e}")
    print(f"  Maximum velocity error: {max_vel_err:.2e}")

    if max_pos_err < 1e-14 and max_vel_err < 1e-14:
        print("  ✅ MACHINE PRECISION - Perfect numerical agreement")
    elif max_pos_err < 1e-10 and max_vel_err < 1e-10:
        print("  ✅ EXCELLENT - Differences within acceptable numerical precision")
    elif max_pos_err < 1e-6 and max_vel_err < 1e-6:
        print("  ⚠️  GOOD - Small differences, likely acceptable for physics")
    else:
        print("  ❌ SIGNIFICANT DIFFERENCES - Investigation required")

print("=" * 60)